<a href="https://colab.research.google.com/github/ZinoTAT/Machine-Learning/blob/master/pytorch/practices/%E4%B8%89%E5%B1%82%E5%85%A8%E8%BF%9E%E6%8E%A5_%E4%BA%A4%E5%8F%89%E7%86%B5%E5%88%86%E7%B1%BBMNIST%E6%95%B0%E6%8D%AE%E9%9B%86-%E6%94%B9%E8%BF%9B(gpu%26%E9%AB%98%E7%BA%A7%E6%8E%A5%E5%8F%A3).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

使用更高级的接口完成了模型，并使用gpu运算，比较遗憾的是移到gpu上之后总体的速度并没有提升，我猜测的原因是数据的运算量本身不大，拷贝到gpu带来的额外时间使得运算速度上的提升被平衡了

In [1]:
import torch 
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
print(torch.__version__)

1.4.0


In [0]:
batch_size = 200
learning_rate=0.01
epochs = 50

In [0]:
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True, transform=transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.1307,), (0.3081,))
    ])),
    batch_size=batch_size, shuffle=True
)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.1307,), (0.3081,))
    ])),
    batch_size=batch_size, shuffle=True
)

In [0]:
class MLP(nn.Module):
  def __init__(self):
    super(MLP,self).__init__()

    self.model = nn.Sequential(
        nn.Linear(784, 200),
        nn.ReLU(inplace=True),
        nn.Linear(200, 200),
        nn.ReLU(inplace=True),
        nn.Linear(200, 10),
        nn.ReLU(inplace=True),
    )
  def forward(self, x):
    x =self.model(x)
    return x

In [5]:
device = torch.device('cuda:0')
print(device)

cuda:0


In [0]:
net = MLP().to(device)
optimizer = optim.SGD(net.parameters(), lr=learning_rate)
criteon = nn.CrossEntropyLoss().to(device)


In [7]:
for epoch in range(epochs):
  for batch_idx, (data, target) in enumerate(train_loader):
    data = data.reshape(-1, 28 * 28)
    data, target = data.to(device), target.to(device)
    logits = net.forward(data)
    loss = criteon(logits, target)
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
  test_loss = 0
  correct = 0
  for data, target in test_loader:
    data = data.view(-1, 28 * 28)
    data, target = data.to(device), target.to(device)
    logits = net.forward(data)
    test_loss += criteon(logits, target).item()
    pred = logits.data.max(1)[1]
    correct += pred.eq(target.data).sum()
  test_loss /= len(test_loader.dataset)
  print('epoch:{}\ntest loss={},correct:{}'.format(epoch, test_loss, float(correct) / float(len(test_loader) * test_loader.batch_size)))
    

epoch:0
test loss=0.005651641857624054,correct:0.6999
epoch:1
test loss=0.003436533486843109,correct:0.8103
epoch:2
test loss=0.0028875512778759,correct:0.8224
epoch:3
test loss=0.0026593067497015,correct:0.8299
epoch:4
test loss=0.0025175923883914947,correct:0.8349
epoch:5
test loss=0.002420681667327881,correct:0.8394
epoch:6
test loss=0.0023372362792491914,correct:0.8426
epoch:7
test loss=0.0022809230625629425,correct:0.8457
epoch:8
test loss=0.0022296066522598267,correct:0.8491
epoch:9
test loss=0.0021802582621574403,correct:0.8504
epoch:10
test loss=0.0021431215018033983,correct:0.8526
epoch:11
test loss=0.0021035252839326857,correct:0.8537
epoch:12
test loss=0.00207072291970253,correct:0.8558
epoch:13
test loss=0.0020416665971279145,correct:0.8574
epoch:14
test loss=0.0020065725654363634,correct:0.8595
epoch:15
test loss=0.0019814818143844603,correct:0.8598
epoch:16
test loss=0.0019574264973402024,correct:0.8602
epoch:17
test loss=0.0019278084456920623,correct:0.8624
epoch:18
test